In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input/feedback-prize-2021/train'):
#     for filename in filenames[0:10]:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls /kaggle/input/feedback-prize-2021

In [ ]:
train = pd.read_csv('/kaggle/input/feedback-prize-2021/train.csv')
train[['discourse_id', 'discourse_start', 'discourse_end']] = train[['discourse_id', 'discourse_start', 'discourse_end']].astype(int)

# the function for building html

In [ ]:
classed_span_format = '<span class="{}">{}</span>'
pre_line_p_format = '<p style="white-space: pre-line">{}</p>'

import html
# parts:[start,end,part_type]
def add_fonts(text, parts):
	full_parts = []
	if parts is None or len(parts) == 0:
		full_parts = [[0, len(text), None]]
	else:
		parts.sort()
		full_parts.append([0, parts[0][0], None])
		for i in range(len(parts)):
			full_parts.append(parts[i])
			if i+1<len(parts):
				full_parts.append([parts[i][1], parts[i+1][0], None])
			else:
				full_parts.append([parts[i][1], len(text), None])
	full_parts_rendered = []
	for part in full_parts:
		text_html = html.escape(text[part[0]:part[1]])
		if part[2] is None:
			full_parts_rendered.append(text_html)
		else:
			full_parts_rendered.append(classed_span_format.format(part[2].lower().replace(' ','_'), text_html))
	html_p = pre_line_p_format.format(''.join(full_parts_rendered))
	# html_p = html_p.replace('\n\n', '</p><p>')
	return html_p

def get_parts(query_id):
	x = train.query('id == "' + query_id + '"')
	parts = [list(x.iloc[i][['discourse_start','discourse_end','discourse_type']]) for i in range(len(x))]
	return parts

def display_text(query_id):
	with open("/kaggle/input/feedback-prize-2021/train/" + query_id + ".txt", 'rt') as f:
		t = f.read()
	return add_fonts(t, get_parts(query_id))

seven discourse type

In [ ]:
discourse_types = list(train['discourse_type'].unique())
print(discourse_types)

import html package in jupyter

In [ ]:
from IPython.core.display import display, HTML

# css style for every discourse type
you may choose your own style

In [ ]:
css_style = """    .lead {
        background-color:#ececea
    }
    .position {
        background-color:yellow;
        font-weight:bold
    }
    .evidence {
        background-color:powderblue
    }
    .claim {
        background-color:#b5c4b1
    }
    .concluding_statement {
        background-color:#ececea;
        text-decoration: underline
    }
    .counterclaim {
        background-color:orange
    }
    .rebuttal {
        background-color:tomato
    }"""

HTML("""
<style>
""" + css_style + """
</style>
""")

random browse slice of filenames

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/feedback-prize-2021/train'):
    for filename in filenames[1001:1010]:
        print(os.path.join(dirname, filename))

# Display the discourse_type in the queried essay

In [ ]:
discourse_types_css = [classed_span_format.format(t.lower().replace(' ', '_'), t) for t in discourse_types]
css_display_html = pre_line_p_format.format('\n'.join(discourse_types_css))
display(HTML(css_display_html))


choose an essay id from the upper browsing result

In [ ]:
query_id= 'AFDB9B105CD1'
get_parts(query_id)

In [ ]:
y = display_text(query_id)
display(HTML(y))
# print(y)

# Output to dir

save css file

In [ ]:
!mkdir /kaggle/working/html_display

In [ ]:
with open('/kaggle/working/html_display/discourse_type_style.css', 'wt') as f:
    f.write(css_style)

save html files

In [ ]:
!mkdir /kaggle/working/html_display/htmls

In [ ]:
html_format = """<!DOCTYPE html>
<html>
<head>
	<title></title>
	<link rel="stylesheet" type="text/css" href="../discourse_type_style.css">
</head>
<body>
HTML_BODY
</body>
</html>"""

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/feedback-prize-2021/train'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        query_id = os.path.splitext(filename)[0]
        y = display_text(query_id)
        if y:
            with open("/kaggle/working/html_display/htmls/" + query_id + ".html", "wt") as f:
                f.write(html_format.replace("HTML_BODY", y))
#             display(HTML(y))

In [ ]:
!zip -r /kaggle/working/html_display.zip /kaggle/working/html_display